In [7]:
import json
file_path="/data/recipe_generation/recipes_v02_251215/recipes_google_gemini-2.5-pro_style_self_select_nutrition.json"
with open(file_path, "r") as f:
    data = json.load(f)
print((data[0].keys()))
print(type(data[0]['recipe']))

dict_keys(['index', 'generated_at', 'input_token_count', 'output_token_count', 'generation_time', 'core_ingredients_list', 'recipe'])
<class 'dict'>


In [13]:
from typing import Dict, Any
def recipe_filter(recipe: Dict[str, Any]) -> Dict[str, Any]|str:
    recipe_text=dict()
    recipe_text['菜名']=recipe['title']
    recipe_text['核心食材']=recipe['ingredients']['core_ingredients']
    recipe_text['其它配料']=recipe['ingredients']['additional_seasonings']

    style = "、".join(recipe['style_select']) + "。"  
    recipe_text['菜系&风格']=style+recipe['ingredients']['cuisine_style']

    recipe_text['制作过程']=dict()
    recipe_text['制作过程']['准备工作']=recipe['preparation']
    recipe_text['制作过程']['烹饪过程']=recipe['cooking_steps']
    recipe_text['制作过程']['tips']=recipe['tips']

    return recipe_text

r=recipe_filter(data[1]['recipe'])
print(r)


{'菜名': '玫瑰琉璃鹅肝佐焦糖苹果', '核心食材': [{'name': '鹅肝/鸭肝', 'amount': '150克'}, {'name': '苹果', 'amount': '1个 (约180克)'}, {'name': '黄油', 'amount': '20克'}, {'name': '玫瑰露酒', 'amount': '30毫升'}], '其它配料': [{'name': '白砂糖', 'amount': '15克'}, {'name': '意大利香醋 (Balsamic Vinegar)', 'amount': '5毫升'}, {'name': '盐', 'amount': '1克'}, {'name': '现磨黑胡椒', 'amount': '少许 (约1克)'}, {'name': '新鲜玫瑰花瓣或百里香', 'amount': '几片 (装饰用)'}], '菜系&风格': '法式、中式-粤菜。融合法式经典煎鹅肝技法与粤式琉璃芡汁的创意。鹅肝的丰腴、苹果的酸甜与玫瑰露的芬芳交织，口感层次丰富，风味中西合璧。', '制作过程': {'准备工作': '1. 准备鹅肝：将冷藏的鹅肝切成约2厘米厚的片状。用厨房纸巾轻轻吸干表面水分。在鹅肝两面撒上极少量的盐和现磨黑胡椒。\n2. 准备苹果：苹果去皮去核，切成约1厘米厚的圆片或月牙片。\n3. 准备酱汁：将玫瑰露酒与意大利香醋混合备用。', '烹饪过程': '1. 焦糖化苹果：取一半黄油（10克）放入平底锅中，中火融化。加入苹果片和白砂糖，小火慢煎，直至两面呈现漂亮的焦糖色且质地变软。盛出苹果片，铺在盘底作为基座。\n2. 香煎鹅肝：将同一平底锅烧热（无需洗锅，也无需额外放油），转中高火。放入鹅肝片，每面快速香煎约45-60秒，直至表面金黄酥脆，内部依然柔软。煎制过程中鹅肝会析出大量油脂。将煎好的鹅肝取出，放置在焦糖苹果片上。\n3. 制作玫瑰琉璃汁：倒掉锅中多余的鹅肝油，只留约一茶匙的底油。转小火，倒入准备好的玫瑰露酒和香醋混合液，快速刮擦锅底的焦香物。酱汁会迅速沸腾并开始变得浓稠，形成一层亮如琉璃的薄芡汁。\n4. 完成与摆盘：立即将烧好的玫瑰琉璃汁淋在鹅肝上。用新鲜的玫瑰花瓣或几叶百里香点缀即可上桌。', 'tips': '1. 鹅肝温度：煎鹅肝前，应使其恢复到室温，这样内外受热更均匀，不易外焦内生

In [ ]:
prompt = f"""
    核心任务: 你将收到一份结构化的菜谱文本。你的任务是根据菜谱内容推理出菜肴成品的外观，然后用一段话详细精准地描述出这道菜的外观。

    转换规则与思维链: 你必须严格按照以下四个步骤思考，并将所有元素融合成
    一个连贯的描述菜肴样貌的提示词。
    1. 核心主体: 首先根据菜谱的内容对这道菜的主体进行精准的描述，你需要根据菜谱上的主体材料、烹饪方法等信息，推理出这道菜的主体样貌。描述要确切，避免使用笼统的术语。
    2. 关键配料与细节: 列举菜谱上可见的成分、配料，忽略像糖分、盐等对菜肴外观没有显著影响的元素。比如“铺满厚切的培根”、“金黄的鱿鱼圈”或“层叠的五层酪乳松饼”。
    3. 摆盘与呈现: 根据菜谱的步骤，推理出食物的摆放方式和盛放的容器。
    4. 摄影风格:指定为“中焦拍摄”、“写实风格”

    注意:
    1.必须完全遵从菜谱的描述，保持"菜谱-提示词输出"的文本一致性。
    2.使用描绘菜品样貌的描述性文本，尽量避免使用描述行为的文本。

    示例：
    1.一款培根奶酪汉堡，第一层为圆形的面包胚，上面撒有黑芝麻，第二层是生菜，第三层是厚切的培根，第四层是融化的陈年切达奶酪，第五层是绿色生菜，第六层也是烤过的黑色芝麻圆形面包；摆放在粗糙的木板上；超写实风格，中焦拍摄。
    2.一款巧克力熔岩蛋糕，融化的黑巧克力从中心流出，蛋糕表面中央点缀以三颗覆盆子和一片薄荷叶，表面均匀轻撒金色糖粉；摆放于洁净的白色瓷盘上；超写实风格，中焦拍摄。

    【菜谱文本】
    "{r}"
"""
print(prompt)

IndentationError: unexpected indent (2190215647.py, line 2)